In [1]:
from Folder import Folder

train_folder = Folder('../data/train.h5',is_test=True, is_eval = False)

print(len(train_folder))

100


In [2]:
##批量化
from torch.utils.data import DataLoader
batch_size = 2
train_loader = DataLoader(train_folder,
                         batch_size=batch_size,
                         num_workers=1,
                         shuffle=False)

In [3]:
for batch_train in train_loader:
    print(batch_train['en_index_list'])
    print(batch_train['en_lengths'])
    break



Columns 0 to 12 
    3     4     5     6     7     8     9    10     1     2     2     2     2
   11    12    13    11    14    15    16    17    18    19    20    21    22

Columns 13 to 25 
    2     2     2     2     2     2     2     2     2     2     2     2     2
    1     2     2     2     2     2     2     2     2     2     2     2     2

Columns 26 to 38 
    2     2     2     2     2     2     2     2     2     2     2     2     2
    2     2     2     2     2     2     2     2     2     2     2     2     2

Columns 39 to 49 
    2     2     2     2     2     2     2     2     2     2     2
    2     2     2     2     2     2     2     2     2     2     2
[torch.LongTensor of size 2x50]


  9
 14
[torch.LongTensor of size 2]



In [4]:
import dataProcess as dp
inputlang = dp.Lang('en')
outputlang = dp.Lang('zh')
inputlang.load('../data/en_input_dict.pkl')
outputlang.load('../data/zh_output_dict.pkl')

print(inputlang.name,inputlang.n_words)
print(outputlang.name,outputlang.n_words)

en 68605
zh 5570


In [5]:
import numpy as np 
import torch
from torch import nn
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import json
import random
import torch.nn.utils.rnn as rnn_utils

In [45]:
class Encoder(nn.Module):
    def __init__(self, embedding_size, en_dims, en_hidden_size, 
                 dropout,is_bidirectional):
        super(Encoder, self).__init__()
        
        
        self.en_hidden_size = en_hidden_size
        
        self.lstm = torch.nn.LSTM(input_size = en_dims,
                                 hidden_size = en_hidden_size,#隐藏层的维数
                                 num_layers = 1,
                                 dropout = dropout,
                                 bidirectional = is_bidirectional,
                                 batch_first = False)
        
        self.dropout = nn.Dropout(dropout)
        #self.use_cuda = use_cuda
        
    def forward(self, inputs, seq_length):
#         inputs = Variable(inputs).long()
#         inputs = inputs.cuda() if self.use_cuda else inputs
        #inputs is len*batch*en_dims
        #inputs = self.embedding(inputs)
        #inputs = torch.transpose(inputs, 0, 1)
        
        
        #要想批量训练，需要pack padded sequence
        packed = rnn_utils.pack_padded_sequence(input=inputs, 
                                                lengths=list(seq_length), 
                                                batch_first = False)
        
        #参数不止一个，不知道是否需要加入？y应该不用，pytorch会自动添加
        packed_out, _ = self.lstm(packed)
        
        unpacked_, _ = rnn_utils.pad_packed_sequence(packed_out)
        
        return unpacked_ #packedSequence(output, a list of lengths of each sequence)
    
class Decoder(nn.Module):
    def __init__(self, embedding_size, zh_dims, en_hidden_size, zh_hidden_size, 
                 dropout, atten_vec_size, use_cuda):
        super(Decoder, self).__init__()
        
        self.use_cuda = use_cuda
        
        self.zh_hidden_size = zh_hidden_size
        self.embedding = torch.nn.Embedding(num_embeddings=embedding_size,
                                           embedding_dim = zh_dims)
        
        self.lstm_cell = torch.nn.LSTMCell(input_size=zh_dims,
                                          hidden_size=zh_hidden_size, 
                                          bias=True)
        
        
#         self.atten_ws = nn.Linear(in_features=zh_hidden_size, 
#                                   out_features=atten_vec_size)
        
#         self.atten_uh = nn.Linear(in_features = en_hidden_size, 
#                                   out_features = atten_vec_size)
        
#         self.atten_v = nn.Linear(in_features = atten_vec_size, 
#                                  out_features = 1)
        
        self.fc = nn.Linear(in_features=zh_hidden_size, 
                            out_features=embedding_size)
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        
    def forward(self, inputs, encoder_hidden_state):
        #inputs should be Length*Batch*en_hidden_size
        #gtruths should be Length*Batch*Features
        inputs = self.embedding(inputs)
        #gtruths = torch.transpose(gtruths, 0, 1)# length batch nfeature#
        
        hx = encoder_hidden_state[-1]#只拿最后一个time step的hidden state
        #hx = self.fc(hx)#暂时让en_hidden_state 与 zh_hidden_state一样大小
        
        if self.use_cuda:
            #hx = Variable(torch.zeros(encoder_hidden_state.size(1), self.zh_hidden_size)).cuda() 
            cx = Variable(torch.zeros(encoder_hidden_state.size(1), self.zh_hidden_size)).cuda()
        else:
            #hx = Variable(torch.zeros(encoder_hidden_state.size(1), self.zh_hidden_size))
            cx = Variable(torch.zeros(encoder_hidden_state.size(1), self.zh_hidden_size))
            
        
        logits = [0 for i in range(inputs.size(0))]
        predicts = [0 for i in range(inputs.size(0))]
        for t in range(inputs.size(0)):   #at each time step t
            hx, cx =lstm_cell(inputs[t],(hx, cx))
            output.append(hx)
            logits[i] = self.fc(hx)#词典中是所要预测的目标的概率
            _, predicts[i] = torch.max(logits[i], 1)
            logits[i] = logits[i].view(1, logits[i].size(0), logits[i].size(1))
            predicts[i] = predicts[i].view(1,predicts[i].size(0))
            
        #predicts = torch.cat(predicts,0)
        return logits, predicts.data.cpu().numpy()
            
        
            
class Seq2Seq(nn.Module):
    def __init__(self, en_dims, zh_dims, en_voc, zh_voc, dropout, en_hidden_size,
                zh_hidden_size, atten_vec_size, use_cuda):
        super(Seq2Seq, self).__init__()
#         self.en_embedding = torch.nn.Embedding(num_embeddings = en_voc,
#                                               embedding_dim = en_dims)

        self.use_cuda = use_cuda
    
        self.en_embedding = torch.nn.Embedding(num_embeddings = en_voc, #英文词典的大小
                                           embedding_dim = en_dims)# 英文词向量的维数
        
        
    
        self.cost_func = nn.CrossEntropyLoss()
        self.encoder = Encoder(embedding_size = en_voc, 
                               en_dims = en_dims, 
                               en_hidden_size=en_hidden_size,
                               dropout = dropout,
                               is_bidirectional = True)
        
        self.decoder = Decoder(embedding_size=zh_voc, 
                              zh_dims = zh_dims, 
                              en_hidden_size = en_hidden_size*2,
                              zh_hidden_size = zh_hidden_size, 
                              atten_vec_size=atten_vec_size, 
                              dropout = dropout,
                              use_cuda = use_cuda)
        
    def order(self, inputs, inputs_len):
        #inputs_len 是重新排序后的结果
        inputs_len, sort_ids = torch.sort(inputs_len, 0, descending=True)
        sort_ids_ = Variable(sort_ids).cuda() if self.use_cuda else Variable(sort_ids)
        #按照下标去排序，保证inputs与inputs_len可以对齐
        inputs = inputs.index_select(0, sort_ids_)
        
        _, true_order_ids = torch.sort(sort_ids, 0, descending=False)
        
        return inputs, inputs_len, true_order_ids
    
    def forward(self, inputs, gtruths, inputs_len):
        if self.use_cuda:
            inputs = Variable(inputs).long().cuda() 
        else:
            inputs = Variable(inputs).long()
            
        
        inputs = self.en_embedding(inputs)
        inputs = inputs.transpose(0,1)
        print('inputs:',inputs)
        ###这里是否需要先转置一下
        
        #change the order according to the lens
        inputs, sorted_len, true_order_ids = self.order(inputs, inputs_len)
        
        encoder_result = self.encoder(inputs, sorted_len)
        
        if self.use_cuda:
            true_order_ids_ = Variable(true_order_ids).cuda() 
        else:
            true_order_ids_ = Variable(true_order_ids)
            
        encoder_result = encoder_result.index_select(0, true_order_ids_)
        
        #logits 是没有经过softmax的
        logits, predict = self.decoder(encoder_result, gtruths)
        
        return logits, predict, encoder_result
    
    def get_loss(self, logits, labels):
        if self.use_cuda:
            labels = Variable(labels).long().cuda() 
        else:
            labels = Variable(labels).long()
            
        labels = labels.transpose(0, 1)
        
        logits = logits.contiguous().view(-1, logits.size(-1))
        labels = labels.contiguous().view(-1)
        
        loss = torch.mean(self.cost_func(logits, labels))
        
        return loss
    
    

In [31]:
def train(lr, net, epoches, train_loader, use_cuda):
    if use_cuda:
        net.cuda()
    
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr = lr)
    
    net.train()
    
    for epoch in range(epoches):
        #批量处理
        for batch_train in train_loader:
            entext = batch_train['en_index_list']
            enlen = batch_train['en_lengths']
            zhgtruth = batch_train['zh_index_list']
            zhlen = batch_train['zh_lengths']
            zhlabel = batch_train['zh_labels_list']
            print('tset')
            logits, predict, encoder_result = net(entext, zhgtruth, enlen)
            loss = net.get_loss(logits, zhlabel)
            
            optimizer.zero_grad()
            loss.bachward()
            optimizer.step()
            
            print('loss:',loss.data)
#         net.eval()
#         logits, predict, encoder_result = net(entext, zhgtruth, enlen)

In [26]:
en_dims = 256
zh_dims = 256
en_hidden_size = 256
zh_hidden_size = 256
atten_vec_size = 256

net = Seq2Seq(en_dims = en_dims,
             zh_dims = zh_dims, 
             en_voc = inputlang.n_words, 
             zh_voc = outputlang.n_words, 
             dropout = 0.1,
             en_hidden_size = en_hidden_size,
             zh_hidden_size = zh_hidden_size,
             atten_vec_size = atten_vec_size,
             use_cuda = False)



In [46]:
train(lr = 0.01, net=net, epoches = 100, train_loader = train_loader,use_cuda = False)

ValueError: lengths array has incorrect size

In [50]:
import torch 
A = torch.randn(6,4,5)
A_len = torch.Tensor([4,3,5])

In [51]:
A_len


 4
 3
 5
[torch.FloatTensor of size 3]

In [ ]:
packed = torch.nn.utils.rnn.pad_packed_sequence(A, )

In [6]:
import numpy as np 
import torch
from torch import nn
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import json
import random
import torch.nn.utils.rnn as rnn_utils

import numpy as np 
import torch
from torch import nn
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import json
import random
import torch.nn.utils.rnn as rnn_utils


In [41]:
class Encoder(nn.Module):
    def __init__(self, use_cuda, en_dims, en_hidden_size, dropout_p, bidirectional=False):

        super(Encoder, self).__init__()

        self.use_cuda = use_cuda

        self.lstm = nn.LSTM(input_size = en_dims,  #输入词向量的维度
                            hidden_size = en_hidden_size,  # hx的维度
                            num_layers = 1,
                            bias = True,
                            batch_first = False,
                            dropout = dropout_p, 
                            bidirectional = bidirectional)#双向lstm

    #inputs 是Len*Batch*en_dims   inputs_len 是记录每个句子的长度(长度是降序的)，它的长度就是batch_size*1大小。
    def forward(self, inputs, inputs_len=None):
        #packed = rnn_utils.pack_padded_sequence(input = inputs,  
        #                               lengths = list(inputs_len))

        #packed_out 是Len*Batch*(en_hidden_size*2)
        #packed_out, _ = self.lstm(packed)

        unpacked, _ = self.lstm(inputs)
        
        #经过unpack之后，unpacked是 maxLen*batch*en_hidden_size*num_bidirection
        #注意maxLen是这个batch中句子最长的那个，长度小的句子会补0
        #num_bidirection 由bidirectional这个参数决定，双向就是hidden_size会翻倍
        #unpacked, _ = rnn_utils.pad_packed_sequence(packed_out)
        
        return unpacked

class Decoder(nn.Module):
    def __init__(self, use_cuda, zh_dims, zh_hidden_size, en_hidden_size, zh_voc):
        super(Decoder, self).__init__()

        self.use_cuda = use_cuda

        self.zh_hidden_size = zh_hidden_size

        self.lstm_cell = nn.LSTMCell(input_size = zh_dims,
                                    hidden_size = zh_hidden_size)

        #加入attention，由于attention有自己的维度，所以需要将hidden_state的维度转换到atten_vec_size
#         self.atten_ws = nn.Linear(in_features = zh_hidden_size,
#                                 out_features = atten_vec_size)
        
        self.en2zh_size = nn.Linear(in_features = en_hidden_size, 
                                    out_features = zh_hidden_size)

        self.hx2zh_voc = nn.Linear(in_features = zh_hidden_size,
                                    out_features = zh_voc)

    #encoder_outputs是maxLen*batch*(en_hidden_size*num_bidirection)
    #inputs就是gtruths，即正确的翻译
    def forward(self, inputs, encoder_outputs):

        hx = encoder_outputs[-1] #只取最后一个输出

        #hx = self.en2zh_size(hx)# batch*zh_hidden_size

        if self.use_cuda:
            #这里是size(0)，感觉不是呀
            #hx = Variable(torch.zeros(encoder_outputs.size(0), self.zh_hidden_size)).cuda()
            cx = Variable(torch.zeros(encoder_outputs.size(1), self.zh_hidden_size)).cuda()
        else:
            #hx = Variable(torch.zeros(encoder_outputs.size(0), self.zh_hidden_size))
            cx = Variable(torch.zeros(encoder_outputs.size(1), self.zh_hidden_size))

        logits = [0 for i in range(inputs.size(0))]
        predicts = [0 for i in range(inputs.size(0))]
        outputs = Variable(torch.zeros(inputs.size(0), encoder_outputs.size(1), self.zh_hidden_size))
        for i in range(inputs.size(0)):
            #print(inputs[i])
            hx, cx = self.lstm_cell(inputs[i], (hx, cx))
            logits[i] = self.hx2zh_voc(hx)
            _, predicts[i] = torch.max(logits[i], 1)#我们要的是最大值对应的下标而已，所以第一个输出不要了

            logits[i] = logits[i].view(1, logits[i].size(0), logits[i].size(1))
            predicts[i] = predicts[i].view(1, predicts[i].size(0))

        return torch.cat(logits), torch.cat(predicts)





class Seq2Seq(nn.Module):
    def __init__(self, use_cuda, en_voc, en_dims, zh_voc, zh_dims, en_hidden_size,
                zh_hidden_size, dropout_p):
        super(Seq2Seq, self).__init__()

        self.use_cuda = use_cuda

        self.en_embedding = nn.Embedding(num_embeddings = en_voc,
                                        embedding_dim = en_dims)

        self.zh_embedding = nn.Embedding(num_embeddings = zh_voc,
                                        embedding_dim = zh_dims)

        self.cost_func = nn.CrossEntropyLoss()

        self.encoder = Encoder(use_cuda = use_cuda,
                                en_dims = en_dims,
                                en_hidden_size = en_hidden_size,
                                dropout_p = dropout_p)

        self.decoder = Decoder(use_cuda = use_cuda, 
                                zh_dims = zh_dims,
                                zh_hidden_size = zh_hidden_size,
                                en_hidden_size = en_hidden_size,
                                zh_voc = zh_voc)

    #句子就以原始形式作为参数
    def forward(self, inputs, gtruths, inputs_len):
        if self.use_cuda:
            inputs = Variable(inputs).long().cuda()
            gtruths = Variable(gtruths).long().cuda()
            inputs = inputs.transpose(0, 1)
            gtruths = gtruths.transpose(0, 1)
        else:
            inputs = Variable(inputs).long()
            gtruths = Variable(gtruths).long()
            inputs = inputs.transpose(0, 1)
            gtruths = gtruths.transpose(0, 1)

        inputs = self.en_embedding(inputs)

        encoder_outputs = self.encoder(inputs)

        gtruths = self.zh_embedding(gtruths)
        #print(type(encoder_outputs))
        logits, predicts = self.decoder(gtruths, encoder_outputs)

        return logits, predicts

    def get_loss(self, logits, labels):
        if self.use_cuda:
            labels = Variable(labels).long().cuda()
            labels = labels.transpose(0, 1)
        else:
            labels = Variable(labels).long()
            labels = labels.transpose(0, 1)

            logits = logits.contiguous().view(-1, logits.size(-1))
            labels = labels.contiguous().view(-1)

            loss = torch.mean(self.cost_func(logits, labels))

            return loss


In [13]:
def train(use_cuda, lr, net, epoches, train_loader): 
    #use_cuda = torch.cuda.is_available()

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr = lr)

    net.train()

    for epoch in range(epoches):
        for data in train_loader:
            entext = data['en_index_list']
            enlen = data['en_lengths']
            zhgtruths = data['zh_index_list']
            zhlen = data['zh_lengths']
            zhlabels = data['zh_labels_list']

            logits, predicts = net(entext, zhgtruths, enlen)

            loss = net.get_loss(logits, zhlabels)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            print('loss',loss)
            
def evaluate(use_cuda, net, eval_data, inputlang, outputlang):
    net.eval()
    for data in eval_data:
        entext = data['en_index_list']
        enlen = data['en_lengths']
        zhgtruths = data['zh_index_list']
        zhlen = data['zh_lengths']
        zhlabels = data['zh_labels_list']

    logits, predicts = net(entext, zhgtruths, enlen)
    
    
    
    
    
    

In [ ]:
en_dims = 256
zh_dims = 256
en_hidden_size = 256
zh_hidden_size = 256

use_cuda = torch.cuda.is_available()
use_cuda = False

net = Seq2Seq(use_cuda = use_cuda,
             en_voc = inputlang.n_words,
             en_dims = en_dims,
             zh_voc = outputlang.n_words,
             zh_dims = zh_dims,
             en_hidden_size = en_hidden_size,
             zh_hidden_size = zh_hidden_size,
             dropout_p = 0)

train(use_cuda=use_cuda, lr = 0.01, net=net, epoches = 5000, train_loader=train_loader)

loss Variable containing:
 8.6210
[torch.FloatTensor of size 1]

loss Variable containing:
 3.3870
[torch.FloatTensor of size 1]

loss Variable containing:
 2.1580
[torch.FloatTensor of size 1]

loss Variable containing:
 4.1592
[torch.FloatTensor of size 1]

loss Variable containing:
 3.2066
[torch.FloatTensor of size 1]

loss Variable containing:
 1.8771
[torch.FloatTensor of size 1]

loss Variable containing:
 1.5989
[torch.FloatTensor of size 1]

loss Variable containing:
 1.8065
[torch.FloatTensor of size 1]

loss Variable containing:
 1.5517
[torch.FloatTensor of size 1]

loss Variable containing:
 2.1177
[torch.FloatTensor of size 1]

loss Variable containing:
 1.2956
[torch.FloatTensor of size 1]

loss Variable containing:
 1.5380
[torch.FloatTensor of size 1]

loss Variable containing:
 1.1162
[torch.FloatTensor of size 1]

loss Variable containing:
 2.0689
[torch.FloatTensor of size 1]

loss Variable containing:
 1.1935
[torch.FloatTensor of size 1]

loss Variable containing:

In [7]:


en_dims = 3
en_hidden_size = 5
zh_dims = 3
zh_hidden_size = 5
batch_size = 2
max_length = 6
inputs = Variable(torch.randn(max_length,batch_size,en_dims))

test_encoder = Encoder(use_cuda=False,
                        en_dims = en_dims,
                        en_hidden_size = en_hidden_size,
                        dropout_p = 0)

test_decoder = Decoder(use_cuda = False,
                      zh_dims = zh_dims,
                      zh_hidden_size = zh_hidden_size,
                      en_hidden_size = en_hidden_size,
                      zh_voc = 10)

encoder_outputs = test_encoder(inputs)
print(encoder_outputs)

logits, predicts = test_decoder()


Variable containing:
(0 ,.,.) = 
 -0.1561  0.1726 -0.0357  0.1376 -0.1111
 -0.1416  0.1337 -0.0016  0.0508 -0.1249

(1 ,.,.) = 
 -0.2469  0.0897  0.0297 -0.0014 -0.1906
 -0.0730  0.0346  0.0542 -0.0564 -0.2169

(2 ,.,.) = 
 -0.1428  0.2959 -0.0666  0.3762 -0.2540
 -0.0130 -0.0633  0.0479 -0.1081 -0.2869

(3 ,.,.) = 
 -0.1591  0.1509 -0.0654  0.2124 -0.2539
 -0.0273  0.1504 -0.0220  0.0143 -0.3143

(4 ,.,.) = 
 -0.1941  0.0078  0.0031  0.0253 -0.2321
 -0.2140  0.1143 -0.0421  0.0800 -0.2250

(5 ,.,.) = 
 -0.3703 -0.1194  0.2797 -0.1027 -0.2114
 -0.3322  0.2050 -0.0620  0.2261 -0.2105
[torch.FloatTensor of size 6x2x5]



In [39]:
tt = torch.randn(2,3).view(1, 2, 3)
tt = [tt, tt, tt]
tt

[
 (0 ,.,.) = 
   0.6888  1.3697 -0.8405
  -1.9755 -0.7241 -1.1595
 [torch.FloatTensor of size 1x2x3], 
 (0 ,.,.) = 
   0.6888  1.3697 -0.8405
  -1.9755 -0.7241 -1.1595
 [torch.FloatTensor of size 1x2x3], 
 (0 ,.,.) = 
   0.6888  1.3697 -0.8405
  -1.9755 -0.7241 -1.1595
 [torch.FloatTensor of size 1x2x3]]

[
  1.3432 -0.0026 -2.3247
  1.1921 -2.0199 -0.9372
 [torch.FloatTensor of size 2x3], 
  1.3432 -0.0026 -2.3247
  1.1921 -2.0199 -0.9372
 [torch.FloatTensor of size 2x3], 
  1.3432 -0.0026 -2.3247
  1.1921 -2.0199 -0.9372
 [torch.FloatTensor of size 2x3]]

In [40]:
ttt = torch.cat(tt)
ttt


(0 ,.,.) = 
  0.6888  1.3697 -0.8405
 -1.9755 -0.7241 -1.1595

(1 ,.,.) = 
  0.6888  1.3697 -0.8405
 -1.9755 -0.7241 -1.1595

(2 ,.,.) = 
  0.6888  1.3697 -0.8405
 -1.9755 -0.7241 -1.1595
[torch.FloatTensor of size 3x2x3]

In [8]:

import importlib
import torch
import Folder
from torch.utils.data import DataLoader
import dataProcess as dp
import transformer
import net

import train

importlib.reload(Folder)
importlib.reload(dp)
importlib.reload(transformer)
importlib.reload(net)
importlib.reload(train)

<module 'train' from '/data/liwb/Documents/projects/machineTranslation/code2/train.py'>

In [2]:



use_cuda = torch.cuda.is_available()

batch_size = 2
en_dims = 256
zh_dims = 256
en_hidden_size = 256
zh_hidden_size = 256



if __name__ == '__main__':
    #train_folder = Folder('../data/train.h5',is_test=False)
    train_folder = Folder.Folder('../data/train.h5',is_test=True, is_eval = False)
    train_loader = DataLoader(train_folder,
                         batch_size=batch_size,
                         num_workers=1,
                         shuffle=False)

    
    inputlang = dp.Lang('en')
    outputlang = dp.Lang('zh')
    inputlang.load('../data/en_input_dict.pkl')
    outputlang.load('../data/zh_output_dict.pkl')

    print(inputlang.name,inputlang.n_words)
    print(outputlang.name,outputlang.n_words)
    
    tf = transformer.Transformer(inputlang, outputlang)
    
    net1 = net.Seq2Seq(use_cuda = use_cuda,
                 en_voc = inputlang.n_words,
                 en_dims = en_dims,
                 zh_voc = outputlang.n_words,
                 zh_dims = zh_dims,
                 en_hidden_size = en_hidden_size,
                 zh_hidden_size = zh_hidden_size,
                 dropout_p = 0)

   

en 68605
zh 5570


In [4]:
train.train(use_cuda=use_cuda, lr = 0.01, net=net1, 
             epoches =10, train_loader=train_loader)

The 0 epoch of loss is Variable containing:
 8.6258
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 3.6286
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 2.2648
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 4.2515
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 3.5662
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 2.6976
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 2.1284
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 2.6613
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 2.3921
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containing:
 3.1401
[torch.cuda.FloatTensor of size 1 (GPU 0)]

The 0 epoch of loss is Variable containi

In [9]:
train.evaluate(use_cuda = use_cuda,net = net1, eval_data = train_loader, transformer = tf)

< what power do you have over her ? 
= 你对她有什么威慑力
> 

   2
   2
   4
   2
   2
   2
   2
  15
   7
  16
 196
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
   2
[torch.LongTensor of size 80]

< i thought if i left the case that she d just tumble out 
= 我以为如果我不拴上箱子她会跌出来
> 

 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
[torch.LongTensor of size 80]

